Alex McRoberts
Exercise 04

Research Question:
How can we operationalize and compare multiple definitions of transit-supportive density (TSD) for Metro station areas in Washington, DC?

This portion of the analysis will focus on:
- Selecting multiple TSD metrics from the literature
- Gathering and preparing relevant data
- Calculating at least one TSD metric (e.g., people + jobs per acre) for all Metro station buffers

In [ ]:
# 1. Load necessary packages

# 2. Load datasets
    # a. WMATA Metrorail station locations
    # b. DC Census tract shapefiles with ACS demographic data
    # c. LEHD LODES employment data

# 3. Project all datasets to a common CRS for Maryland.

# 4. Create 0.5-mile buffers around each Metro station

# 5. Spatial join Census tracts and/or LODES data to buffers

# 6. Aggregate population, housing units, and jobs within each buffer

# 7. Calculate:
    # - Population density (people per acre)
    # - Jobs density (jobs per acre)
    # - Combined people + jobs per acre

# 8. Compare initial results across stations

# 9. Visualize initial density distributions


Transit-Supportive Density Metrics

Population Density (residents per acre)	TCRP Report 102	
    ~12–15 DU/acre supports frequent bus; 20+ DU/acre for rail
Combined Population + Jobs Density	TCRP Report 128
    45 people + jobs per acre supports light rail/BRT
Net Residential Density (excluding roads, parks) Urban Land Institute
    Focuses on usable land
Floor Area Ratio (FAR)	EPA TOD Guidelines
    ~1.0 FAR for light rail TOD
Walkshed Accessibility (Isochrone Analysis)	ITDP TOD Standard
    Access to 80% of daily needs within 15-minute walk

In [ ]:
# 1. Import packages
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os

# 2. File Paths
station_fp = "Metro_Stations_in_DC.geojson"
tracts_fp = "data/dc_census_tracts.geojson"
acs_fp = "data/acs_population_data.csv"
lodes_fp = "data/lehd_lodes_dc.csv"

# 3. Load data
stations = gpd.read_file(station_fp)
tracts = gpd.read_file(tracts_fp)
acs = pd.read_csv(acs_fp)
lodes = pd.read_csv(lodes_fp)

# 4. CRS Check - Project everything to Maryland State Plane (EPSG:26985)
stations = stations.to_crs(epsg=26985)
tracts = tracts.to_crs(epsg=26985)

In [ ]:
# 5. Merge ACS data into Census tracts
tracts = tracts.merge(acs, on="GEOID")

# Merge in jobs data (LODES)
tracts = tracts.merge(lodes, on="GEOID")

In [ ]:
# 6. Create 0.5-mile buffers around Metro stations
# 0.5 miles = 804.672 meters
stations["geometry"] = stations.buffer(804.672)

In [ ]:
# 7. Spatial Join: Find which tracts intersect each station buffer
# This is a one-to-many relationship: many tracts may touch one station buffer
join = gpd.sjoin(tracts, stations, predicate="intersects")

In [ ]:
# 8. Aggregate population and jobs for each station
# Assume ACS table has 'population' field and LODES table has 'jobs' field
station_summary = join.groupby('index_right').agg({
    'population': 'sum',
    'jobs': 'sum',
    'geometry': 'first'  # Keep buffer geometry for plotting
}).reset_index()

In [ ]:
# 9. Calculate combined people + jobs per acre
# First calculate buffer area in acres
station_summary = gpd.GeoDataFrame(station_summary, geometry='geometry', crs=stations.crs)
station_summary["area_acres"] = station_summary.geometry.area / 4046.86  # 1 acre = 4046.86 m^2

# Calculate density
station_summary["people_jobs_per_acre"] = (station_summary["population"] + station_summary["jobs"]) / station_summary["area_acres"]

In [ ]:
# 10. Simple Visualization
plt.figure(figsize=(10,6))
plt.hist(station_summary["people_jobs_per_acre"], bins=20, edgecolor="black")
plt.title("Distribution of People + Jobs per Acre around Metro Stations (0.5-mile buffer)")
plt.xlabel("People + Jobs per Acre")
plt.ylabel("Number of Stations")
plt.grid(True)
plt.show()